# An approximate solver for Burgers' equation

In this chapter we return to the inviscid Burgers' equation:

\begin{align} \label{burgers}
q_t + \left(\frac{1}{2}q^2\right)_x & = 0.
\end{align}

Although it is trivial to solve the Riemann problem for \eqref{burgers} exactly, it is nevertheless interesting to consider approximate solvers because a numerical scheme does not make use of the full Riemann solution.  Furthermore, Burgers' equation provides a simple setting in which some of the intricacies of more complicated approximate solvers can be introduced.

## How numerical schemes use the Riemann solution
Although we have gone to great pains to determine the full details of each Riemann solution, much of this information is not used in a numerical implementation.  Typically a numerical scheme uses only the value of the flux at $x=0$ to advance the solution.  Equivalently, we can think of the scheme as incorporating only the net impact of all waves traveling to the right (or left) on the adjacent grid points or cells.  It's therefore not useful to consider the internal structure of a rarefaction wave, but rather only the net change in the solution that the rarefaction wave leads to.  Hence our approximate solutions can be thought of as consisting entirely of traveling discontinuities.

## Shock wave solutions

Recall that the exact Riemann solution consists of a single shock or rarefaction wave.  We have a shock if $q_l>q_r$ and we have a rarefaction if $q_l < q_r$.  In the case of a shock wave, we can simply use the exact solution as our approximation.  We have a single wave of strength $q_r-q_l$ traveling at speed $s=(q_r+q_l)/2$.  In terms of fluxes, the numerical flux is $F=f(q_l)$ if $s>0$ and $F=f(q_r)$ if $s<0$.  In the special case $s=0$ we have a stationary shock, and it must be that $f(q_l)=f(q_r) (=F)$. 

## Rarefaction wave solutions

As discussed above, for numerical purposes it is convenient to approximate a rarefaction wave by a traveling discontinuity.  For Burgers' equation this may seem unnecessary, but for more complicated solvers for systems of equations it will be essential.

We will approximate the effect of the rarefaction wave by a fictitious shock, with jump $q_r-q_l$ and speed given by the Rankine-Hugoniot jump condition:

$$s = \frac{f(q_r)-f(q_l)}{q_r-q_l} = \frac{q_r + q_l}{2}.$$

This fictitious shock is not entropy-satisfying, but as long as it's traveling entirely to the left or entirely to the right, the effect on the numerical solution will be the same as if we used a (entropy-satisfying) rarefaction wave.

The numerical flux is again $F=f(q_l)$ if $s>0$ and $F=f(q_r)$ if $s<0$.

## Transonic rarefactions

In all the approximations above, the solution was approximated by a single wave traveling either to the left or right.  However, if $q_l < 0 < q_r$, then the true solution is a transonic rarefaction, in which part of the wave travels to the left and part travels to the right.  In a numerical scheme 